# Hello World, Amazon SageMaker Feature Store
This notebook provides a demo of how easy it is to use SageMaker Feature Store. It does this by leveraging a simple  set of utility functions that wrap the feature store API to keep it simple for a data scientist using Python.

### A few imports

In [1]:
from FeatureStore import Utils
from IPython.core.display import display, HTML
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier

FG_NAME = 'customers'

#### Install pyarrow for reading a sample parquet file from the offline store

In [2]:
!pip install pyarrow

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Create some quick and dirty test data

In [3]:
df = pd.read_csv('customers.csv')
ORIGINAL_RECORD_COUNT = df.shape[0]
df.head()

,Id,UpdateTime,ZipCode,Persona,Churn
0,1,2020-02-01T00:00:00Z,11111,1,1
1,2,2020-02-01T00:00:00Z,11111,1,1
2,3,2020-02-01T00:00:00Z,11111,1,1
3,4,2020-02-01T00:00:00Z,11111,2,0
4,5,2020-02-01T00:00:00Z,11111,2,0


### Delete my existing feature group, since we're just playing in a test environment
For completeness, it is good to delete the Glue table that was created on your behalf by SageMaker Feature Store. See the Glue console [here](https://console.aws.amazon.com/glue).

In [4]:
Utils.delete_feature_group(FG_NAME)

Deleting all s3 objects in prefix: offline-store/355151823911/sagemaker/us-east-1/offline-store/customers in bucket sagemaker-us-east-1-355151823911
Waiting for Feature Group Deletion


### Create a brand new feature group, directly from my dataframe

In [5]:
tags = {'Environment': 'DEV', 
        'CostCenter': 'C20', 
        'Maintainer': 'John Smith', 
        'DocURL': 'https://www.google.com'}
Utils.create_fg_from_df(FG_NAME, df, tags=tags)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup customers successfully created.


### See that it actually worked

In [6]:
Utils.list_feature_groups('cust')

[{'FeatureGroupName': 'customers',
  'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/customers',
  'CreationTime': datetime.datetime(2021, 2, 11, 17, 27, 12, 935000, tzinfo=tzlocal()),
  'FeatureGroupStatus': 'Created'}]

In [7]:
Utils.describe_feature_group(FG_NAME)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/customers',
 'FeatureGroupName': 'customers',
 'RecordIdentifierFeatureName': 'Id',
 'EventTimeFeatureName': 'UpdateTime',
 'FeatureDefinitions': [{'FeatureName': 'Id', 'FeatureType': 'Integral'},
  {'FeatureName': 'UpdateTime', 'FeatureType': 'String'},
  {'FeatureName': 'ZipCode', 'FeatureType': 'Integral'},
  {'FeatureName': 'Persona', 'FeatureType': 'Integral'},
  {'FeatureName': 'Churn', 'FeatureType': 'Integral'}],
 'CreationTime': datetime.datetime(2021, 2, 11, 17, 27, 12, 935000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-355151823911/offline-store'},
  'DisableGlueTableCreation': False,
  'DataCatalogConfig': {'TableName': 'customers-1613064432',
   'Catalog': 'AwsDataCatalog',
   'Database': 'sagemaker_featurestore'}},
 'RoleArn': 'arn:aws:iam::355151823911:role/service-role/AmazonSageMaker-Execut

In [8]:
doc_url = Utils.get_tags(FG_NAME)['DocURL']
print(f'Docs for feature group "{FG_NAME}" is here: {doc_url}')

Docs for feature group "customers" is here: https://www.google.com


### Ingest the features from my dataframe into my new feature group

In [9]:
Utils.ingest_from_df(FG_NAME, df)

### Show that we can lookup the latest feature values

In [10]:
Utils.get_latest_feature_values(FG_NAME, [4,2,6])

[{'Id': 4,
  'UpdateTime': '2020-02-01T00:00:00Z',
  'ZipCode': 11111,
  'Persona': 2,
  'Churn': 0},
 {'Id': 2,
  'UpdateTime': '2020-02-01T00:00:00Z',
  'ZipCode': 11111,
  'Persona': 1,
  'Churn': 1},
 {'Id': 6,
  'UpdateTime': '2020-02-01T00:00:00Z',
  'ZipCode': 11111,
  'Persona': 2,
  'Churn': 0}]

## Show that we can get the history of feature values
The offline store is append-only. New records are added.

#### Now, ingest some new data with later event timestamps
We'll put in two new sets of records each with the event timestamp advanced one day, and the zipcode changed. We should now have three total sets of records:

1. Original, event timestamp Feb 1, zip code 11111
2. New set, with event timestamp Feb 2, zip code 22222
3. Final set, with event timestamp Feb 3, zip code 33333

In [11]:
df['UpdateTime'] = '2020-02-02T00:00:00Z'
df['ZipCode'] = '22222'
Utils.ingest_from_df(FG_NAME, df)

df['UpdateTime'] = '2020-02-03T00:00:00Z'
df['ZipCode'] = '33333'
Utils.ingest_from_df(FG_NAME, df)

#### Look up the full history for a few id's
Wait a few minutes (up to 15) for the data to be there.

In [12]:
ids = [5,6]
features = ['*'] 

mins = 0
while True:
    hist_df = Utils.get_historical_offline_feature_values(FG_NAME, record_ids=ids, feature_names=features)
    if hist_df.shape[0] < ORIGINAL_RECORD_COUNT:
        print('Waiting for offline store data...')
        time.sleep(60)
        mins += 1
    else:
        break

print(f'\nData is available. Waited {mins} minutes\n')
hist_df.sort_values(by=['id','zipcode']).head(30)

Running query:
 SELECT * FROM "customers-1613064432"  WHERE Id IN (5,6)
Waiting for offline store data...
Running query:
 SELECT * FROM "customers-1613064432"  WHERE Id IN (5,6)
Waiting for offline store data...
Running query:
 SELECT * FROM "customers-1613064432"  WHERE Id IN (5,6)
Waiting for offline store data...
Running query:
 SELECT * FROM "customers-1613064432"  WHERE Id IN (5,6)
Waiting for offline store data...
Running query:
 SELECT * FROM "customers-1613064432"  WHERE Id IN (5,6)
Waiting for offline store data...
Running query:
 SELECT * FROM "customers-1613064432"  WHERE Id IN (5,6)
Waiting for offline store data...
Running query:
 SELECT * FROM "customers-1613064432"  WHERE Id IN (5,6)
Waiting for offline store data...
Running query:
 SELECT * FROM "customers-1613064432"  WHERE Id IN (5,6)

Data is available. Waited 7 minutes



,id,updatetime,zipcode,persona,churn,write_time,api_invocation_time,is_deleted
4,5,2020-02-01T00:00:00Z,11111,2,0,2021-02-11 17:33:46.578,2021-02-11 17:27:25.000,False
5,5,2020-02-02T00:00:00Z,22222,2,0,2021-02-11 17:33:46.587,2021-02-11 17:27:25.000,False
0,5,2020-02-03T00:00:00Z,33333,2,0,2021-02-11 17:33:46.587,2021-02-11 17:27:25.000,False
2,6,2020-02-01T00:00:00Z,11111,2,0,2021-02-11 17:33:46.912,2021-02-11 17:27:25.000,False
3,6,2020-02-02T00:00:00Z,22222,2,0,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False
1,6,2020-02-03T00:00:00Z,33333,2,0,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False


#### Browse the set of offline store files in the S3 console

In [13]:
s3_console_url = Utils.get_offline_store_url(FG_NAME)
print(f'Review offline store partitioned data files here: {s3_console_url}')

Review offline store partitioned data files here: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-355151823911?region=us-east-1&prefix=offline-store/355151823911/sagemaker/us-east-1/offline-store/customers/data/


#### See the Glue table that can be used for Athena queries

In [14]:
glue_console_url = Utils.get_glue_table_url(FG_NAME)
print(f'To see the Glue table that was created for you, go here: {glue_console_url}')

To see the Glue table that was created for you, go here: https://console.aws.amazon.com/glue/home?region=us-east-1#table:catalog=355151823911;name=customers-1613064432;namespace=sagemaker_featurestore


#### Examine contents of a sample offline store file

In [15]:
sample_filename = Utils.download_sample_offline_file(FG_NAME)
print(f'Downloaded sample file from offline store: {sample_filename}')
sample_df = pd.read_parquet(sample_filename)
sample_df.head()

Downloaded sample file from offline store: 20200201T000000Z_5lRSAkieCeFIBG51.parquet


,Id,UpdateTime,ZipCode,Persona,Churn,write_time,api_invocation_time,is_deleted
0,2,2020-02-01T00:00:00Z,11111,1,1,2021-02-11 17:33:17.549000+00:00,2021-02-11 17:27:25+00:00,False


#### See the total record count in the offline store for this feature group

In [16]:
total_record_count = Utils.get_historical_record_count(FG_NAME)
print(f'Found {total_record_count:,d} records in "{FG_NAME}" feature group.')

Found 18 records in "customers" feature group.


#### Here we retrieve the full history for all id's

In [17]:
hist_df = Utils.get_historical_offline_feature_values(FG_NAME)
hist_df.sort_values(by=['id','zipcode']).head(100)

Running query:
 SELECT * FROM "customers-1613064432" 


,id,updatetime,zipcode,persona,churn,write_time,api_invocation_time,is_deleted
15,1,2020-02-01T00:00:00Z,11111,1,1,2021-02-11 17:33:46.912,2021-02-11 17:27:24.000,False
0,1,2020-02-02T00:00:00Z,22222,1,1,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False
3,1,2020-02-03T00:00:00Z,33333,1,1,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False
7,2,2020-02-01T00:00:00Z,11111,1,1,2021-02-11 17:33:17.549,2021-02-11 17:27:25.000,False
2,2,2020-02-02T00:00:00Z,22222,1,1,2021-02-11 17:33:17.567,2021-02-11 17:27:25.000,False
13,2,2020-02-03T00:00:00Z,33333,1,1,2021-02-11 17:33:17.576,2021-02-11 17:27:25.000,False
12,3,2020-02-01T00:00:00Z,11111,1,1,2021-02-11 17:33:17.563,2021-02-11 17:27:25.000,False
10,3,2020-02-02T00:00:00Z,22222,1,1,2021-02-11 17:33:17.571,2021-02-11 17:27:25.000,False
11,3,2020-02-03T00:00:00Z,33333,1,1,2021-02-11 17:33:17.579,2021-02-11 17:27:25.000,False
5,4,2020-02-01T00:00:00Z,11111,2,0,2021-02-11 17:33:46.578,2021-02-11 17:27:24.000,False


#### Now let's see what the online store thinks are the latest values

In [18]:
Utils.get_latest_feature_values(FG_NAME, [4,2,6])

[{'Id': 4,
  'UpdateTime': '2020-02-03T00:00:00Z',
  'ZipCode': 33333,
  'Persona': 2,
  'Churn': 0},
 {'Id': 2,
  'UpdateTime': '2020-02-03T00:00:00Z',
  'ZipCode': 33333,
  'Persona': 1,
  'Churn': 1},
 {'Id': 6,
  'UpdateTime': '2020-02-03T00:00:00Z',
  'ZipCode': 33333,
  'Persona': 2,
  'Churn': 0}]

## Train a simple model with features extracted from the feature store
For our example, the dataset we want is the latest values for 3 specific features for each record id.

In [19]:
full_df = Utils.get_latest_offline_feature_values(FG_NAME, feature_names=['ZipCode','Persona','Churn'])
full_df.head(10)

Running query:
 SELECT ZipCode,Persona,Churn FROM (SELECT *, dense_rank() OVER (PARTITION BY Id ORDER BY UpdateTime DESC, Api_Invocation_Time DESC, write_time DESC) AS rank FROM "customers-1613064432" ) WHERE rank = 1 AND NOT is_deleted


,ZipCode,Persona,Churn
0,33333,2,0
1,33333,2,0
2,33333,1,1
3,33333,1,1
4,33333,2,0
5,33333,1,1


In [20]:
train_df = full_df[0:4]
test_df = full_df[4:6]

In [21]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_df[['ZipCode','Persona']], train_df[['Churn']].values.ravel())
clf.predict(test_df[['ZipCode','Persona']])

array([0, 1])

## Demonstrate time travel

#### Time travel to get dataset as it looked on 2020-02-02

In [22]:
Utils.get_offline_feature_values_as_of(FG_NAME, '2020-02-02T00:00:00Z')

Running query:
 SELECT * FROM (SELECT *, dense_rank() OVER (PARTITION BY Id ORDER BY UpdateTime DESC, Api_Invocation_Time DESC, write_time DESC) AS rank FROM "customers-1613064432" WHERE UpdateTime <= '2020-02-02T00:00:00Z') WHERE rank = 1 AND NOT is_deleted


,id,updatetime,zipcode,persona,churn,write_time,api_invocation_time,is_deleted,rank
0,1,2020-02-02T00:00:00Z,22222,1,1,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False,1
1,4,2020-02-02T00:00:00Z,22222,2,0,2021-02-11 17:33:46.587,2021-02-11 17:27:25.000,False,1
2,5,2020-02-02T00:00:00Z,22222,2,0,2021-02-11 17:33:46.587,2021-02-11 17:27:25.000,False,1
3,2,2020-02-02T00:00:00Z,22222,1,1,2021-02-11 17:33:17.567,2021-02-11 17:27:25.000,False,1
4,3,2020-02-02T00:00:00Z,22222,1,1,2021-02-11 17:33:17.571,2021-02-11 17:27:25.000,False,1
5,6,2020-02-02T00:00:00Z,22222,2,0,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False,1


#### Time travel to get dataset as it looked on 2020-02-01

In [23]:
Utils.get_offline_feature_values_as_of(FG_NAME, '2020-02-01T00:00:00Z')

Running query:
 SELECT * FROM (SELECT *, dense_rank() OVER (PARTITION BY Id ORDER BY UpdateTime DESC, Api_Invocation_Time DESC, write_time DESC) AS rank FROM "customers-1613064432" WHERE UpdateTime <= '2020-02-01T00:00:00Z') WHERE rank = 1 AND NOT is_deleted


,id,updatetime,zipcode,persona,churn,write_time,api_invocation_time,is_deleted,rank
0,6,2020-02-01T00:00:00Z,11111,2,0,2021-02-11 17:33:46.912,2021-02-11 17:27:25.000,False,1
1,3,2020-02-01T00:00:00Z,11111,1,1,2021-02-11 17:33:17.563,2021-02-11 17:27:25.000,False,1
2,2,2020-02-01T00:00:00Z,11111,1,1,2021-02-11 17:33:17.549,2021-02-11 17:27:25.000,False,1
3,1,2020-02-01T00:00:00Z,11111,1,1,2021-02-11 17:33:46.912,2021-02-11 17:27:24.000,False,1
4,4,2020-02-01T00:00:00Z,11111,2,0,2021-02-11 17:33:46.578,2021-02-11 17:27:24.000,False,1
5,5,2020-02-01T00:00:00Z,11111,2,0,2021-02-11 17:33:46.578,2021-02-11 17:27:25.000,False,1


## Delete a record and show how it impacts lookups (latest, and historical)

In [24]:
Utils.delete_record(FG_NAME, 1, '2020-02-03T00:00:00Z')

#### After a record is deleted, the online store will no longer return features for that id

In [25]:
Utils.get_latest_feature_values(FG_NAME, [1])

[]

#### Once the deletion is propagated to the offline store, a new record is added with the is_deleted flag set to True
If you get feature values as of that timestamp, the deleted record will be filtered out.

In [26]:
mins = 0
while True:
    hist_df = Utils.get_offline_feature_values_as_of(FG_NAME, '2020-02-03T00:00:00Z')
    if hist_df.shape[0] == ORIGINAL_RECORD_COUNT:
        print('Waiting for offline store deletion update...')
        time.sleep(60)
        mins += 1
    else:
        break

print(f'\nDeletion was propagated. Waited {mins} minutes\n')
hist_df.head(10)

Running query:
 SELECT * FROM (SELECT *, dense_rank() OVER (PARTITION BY Id ORDER BY UpdateTime DESC, Api_Invocation_Time DESC, write_time DESC) AS rank FROM "customers-1613064432" WHERE UpdateTime <= '2020-02-03T00:00:00Z') WHERE rank = 1 AND NOT is_deleted
Waiting for offline store deletion update...
Running query:
 SELECT * FROM (SELECT *, dense_rank() OVER (PARTITION BY Id ORDER BY UpdateTime DESC, Api_Invocation_Time DESC, write_time DESC) AS rank FROM "customers-1613064432" WHERE UpdateTime <= '2020-02-03T00:00:00Z') WHERE rank = 1 AND NOT is_deleted
Waiting for offline store deletion update...
Running query:
 SELECT * FROM (SELECT *, dense_rank() OVER (PARTITION BY Id ORDER BY UpdateTime DESC, Api_Invocation_Time DESC, write_time DESC) AS rank FROM "customers-1613064432" WHERE UpdateTime <= '2020-02-03T00:00:00Z') WHERE rank = 1 AND NOT is_deleted
Waiting for offline store deletion update...
Running query:
 SELECT * FROM (SELECT *, dense_rank() OVER (PARTITION BY Id ORDER BY Upd

,id,updatetime,zipcode,persona,churn,write_time,api_invocation_time,is_deleted,rank
0,4,2020-02-03T00:00:00Z,33333,2,0,2021-02-11 17:33:46.587,2021-02-11 17:27:25.000,False,1
1,5,2020-02-03T00:00:00Z,33333,2,0,2021-02-11 17:33:46.587,2021-02-11 17:27:25.000,False,1
2,6,2020-02-03T00:00:00Z,33333,2,0,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False,1
3,3,2020-02-03T00:00:00Z,33333,1,1,2021-02-11 17:33:17.579,2021-02-11 17:27:25.000,False,1
4,2,2020-02-03T00:00:00Z,33333,1,1,2021-02-11 17:33:17.576,2021-02-11 17:27:25.000,False,1


#### If you retrieve all feature records, you can see that the offline store is an append-only store
A record is added with features written as NaN, with the `is_deleted` flag set to `True`.
The new deleted record has the more recent write_time. 

In [27]:
hist_df = Utils.get_historical_offline_feature_values(FG_NAME)
hist_df.sort_values(by=['id','zipcode']).head(100)

Running query:
 SELECT * FROM "customers-1613064432" 


,id,updatetime,zipcode,persona,churn,write_time,api_invocation_time,is_deleted
11,1,2020-02-01T00:00:00Z,11111.0,1.0,1.0,2021-02-11 17:33:46.912,2021-02-11 17:27:24.000,False
12,1,2020-02-02T00:00:00Z,22222.0,1.0,1.0,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False
2,1,2020-02-03T00:00:00Z,33333.0,1.0,1.0,2021-02-11 17:33:46.913,2021-02-11 17:27:25.000,False
4,1,2020-02-03T00:00:00Z,NaN,NaN,NaN,2021-02-11 17:39:51.197,2021-02-11 17:34:59.000,True
9,2,2020-02-01T00:00:00Z,11111.0,1.0,1.0,2021-02-11 17:33:17.549,2021-02-11 17:27:25.000,False
8,2,2020-02-02T00:00:00Z,22222.0,1.0,1.0,2021-02-11 17:33:17.567,2021-02-11 17:27:25.000,False
1,2,2020-02-03T00:00:00Z,33333.0,1.0,1.0,2021-02-11 17:33:17.576,2021-02-11 17:27:25.000,False
5,3,2020-02-01T00:00:00Z,11111.0,1.0,1.0,2021-02-11 17:33:17.563,2021-02-11 17:27:25.000,False
18,3,2020-02-02T00:00:00Z,22222.0,1.0,1.0,2021-02-11 17:33:17.571,2021-02-11 17:27:25.000,False
0,3,2020-02-03T00:00:00Z,33333.0,1.0,1.0,2021-02-11 17:33:17.579,2021-02-11 17:27:25.000,False


#### Now see what would be retrieved for a training dataset with only the latest values for each id
Notice that the deleted record is not returned.

In [28]:
Utils.get_latest_offline_feature_values(FG_NAME, feature_names=['ZipCode','Persona','Churn'])

Running query:
 SELECT ZipCode,Persona,Churn FROM (SELECT *, dense_rank() OVER (PARTITION BY Id ORDER BY UpdateTime DESC, Api_Invocation_Time DESC, write_time DESC) AS rank FROM "customers-1613064432" ) WHERE rank = 1 AND NOT is_deleted


,ZipCode,Persona,Churn
0,33333,2,0
1,33333,2,0
2,33333,1,1
3,33333,1,1
4,33333,2,0


## Clean up
Delete the feature group and its offline storage.

In [29]:
#Utils.delete_feature_group(FG_NAME)